In [ ]:
!pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

# Launch Phoenix

In [ ]:
import phoenix as px

session = px.launch_app()

# Set Up OTel

In [ ]:
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

tracer_provider = trace_sdk.TracerProvider()
span_exporter = OTLPSpanExporter("http://localhost:6006/v1/traces")
span_processor = SimpleSpanProcessor(span_exporter)
tracer_provider.add_span_processor(span_processor)
trace_api.set_tracer_provider(tracer_provider)

# Instrument CrewAI

In [ ]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True)

# Keys

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)






In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

In [ ]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    # You can pass an optional llm attribute specifying what model you wanna use.
    # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
    tools=[search_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer], tasks=[task1, task2], verbose=1, process=Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

WARNI [opentelemetry.trace] Overriding of current TracerProvider is not allowed


 [2024-08-23 05:09:09][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-08-23 05:09:09][INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather detailed information on key trends, breakthrough technologies, and potential industry impacts. The best way to start is by conducting a thorough internet search on this topic.

Action: Search the internet
Action Input: {"search_query": "latest advancements in AI 2024 key trends breakthrough technologies industry impacts"} 


Search results: Title: 10 Breakthrough Technologies 2024 | MIT Technology Review
Link: https://www.technologyreview.com/2024/01/08/1085094/10-breakthrough-technologies-2024/
Snippet: Every year, we look for promising technologies poised to have a real impa

WARNI [opentelemetry.attributes] Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


**Comprehensive Analysis of the Latest Advancements in AI in 2024**

### Key Trends
1. **AI-Powered Scientific Discovery**:
   - AI systems are increasingly being used to accelerate scientific research.
   - AI algorithms are helping identify new compounds, materials, and solutions faster than traditional methods.

2. **Natural Language Processing (NLP)**:
   - Significant improvements in understanding and generating human language.
   - Enhanced AI models like GPT-4 are more capable of context-aware conversations and content generation.

3. **AI-Powered Automation**:
   - Increased adoption of AI in automating complex tasks across various industries.
   - From manufacturing to customer service, AI is streamlining operations and improving efficiency.

4. **Ethical and Responsible AI**:
   - Greater focus on developing AI systems that are ethical, fair, and transparent.
   - Implementation of regulations to ensure AI systems do not perpetuate biases and are accountable.

5. **AI in Heal

WARNI [opentelemetry.attributes] Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


Thought: I now can give a great answer
Final Answer: 

### The Most Significant AI Advancements in 2024

Artificial Intelligence (AI) is continuing to revolutionize various industries in 2024, bringing about innovations that are reshaping the technological landscape. Here, we explore the key trends, breakthrough technologies, and potential impacts of the latest AI advancements in 2024. Buckle up, because the future is now!

#### Key Trends in AI for 2024

1. **Generative AI Expansion**
   Generative AI models like GPT-4 and beyond have reached new heights, creating highly realistic images, videos, and even fully immersive virtual environments. These advancements are transforming creative fields such as gaming, film, and advertising by automating content creation and enriching user experiences.

2. **Natural Language Processing (NLP) Advancements**
   NLP technologies have made significant strides in understanding and generating human language. Enhanced language models are being leverag

# View Results

This guide is packed in a neat notebook for convenience, however many people will host Arize Phoenix in a [Docker](https://docs.arize.com/phoenix/deployment/docker) Container or Spin up an instance on their local machine.

In [ ]:
session.view()

 # ⭐⭐⭐ If you like this guide, please give [Arize Phoenix](https://github.com/Arize-ai/phoenix) a star ⭐⭐⭐